In [1]:
using DifferentialEquations
using PyPlot
using Random

In [2]:
#Cavity with moving atoms

In [3]:
function photnumbevo(N, K, σv, i)

κ=1
γ=0
Δ=0
g=2*1.36*0.001
ν=0.5;  
    
   
function G(x)
    λ=1
    k=(2*pi)/λ
    return g*cos(k*x)
end
       
x0=[2*pi*j/N for j=0:(N-1)]
Random.seed!(i)
v0=[randn()*σv for j=1:N]

#the DiffEq slightly differs from the paper
#as we use different time scales
#and the equation for <σ^z> instead of <σ^ee>
function cavityall(du, u, p, t)  
    scaling=K
    GU = 0
    for i=1:N
        GU+=G(x0[i]+t*v0[i])*u[i+1]
    end
    #<a>
    du[1]=-κ*u[1]+  scaling * GU
    #<σ->
    for i=2:(N+1)
        du[i]=(im*Δ-γ-ν)*u[i]+G(x0[i-1]+t*v0[i-1])*u[N+i]*u[1]
    end
    #<σ^z>
    for i=(N+2):(2*N+1)
        du[i]=-2*(γ+ν)*u[i]-2*G(x0[i-1-N]+t*v0[i-1-N])*(conj(u[1])*u[i-N]+conj(u[i-N])*u[1])+2*(ν-γ)
    end
end

s0=[0.01+0.0*im for i=1:N]
z0=[-1.0+0.0*im for i=1:N];
u0=[0.0+0.0*im; s0; z0];

tspan=(0.0, 200.0)
prob=ODEProblem(cavityall, u0, tspan)
sol=solve(prob, saveat=0.1);

L=length(sol.t)
expn=abs2.(sol[1, :])

return expn
end

photnumbevo (generic function with 1 method)

In [4]:
tarray=collect(0:0.1:200);

In [5]:
rc("text",  usetex="True")
rc("font", family="serif")
rc("font", size=38)

In [6]:
########################################################################################################
# (1) plots

In [7]:
result1=photnumbevo(1000, 1000, 0.0, 1);
result2=photnumbevo(1000, 1000, 0.2, 1);
result3=photnumbevo(1000, 1000, 0.4, 1);

In [8]:
result4=photnumbevo(200, 5000, 0.0, 1);
result5=photnumbevo(200, 5000, 0.2, 1);
result6=photnumbevo(200, 5000, 0.4, 1);

In [10]:
#paper plot
pygui(true)
scale=1000;
figure("example trajectories", figsize=(24,10))
subplot(121)
plot(2*tarray, result4/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.0")
plot(2*tarray, result5/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.1")
plot(2*tarray, result6/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.2")
#title("Time evolution for 200 Atom clusters")
vlines(2*100, 0, 280000/scale, color="r", linestyle="dotted")
arrow(200, 210,100,0; length_includes_head=true, color="red", width=1, head_width=8, head_length=10)
text(210, 220, "quasi-steady-state", color="r")

xlabel(L"\kappa t")
ylabel(L"\mathrm{photon\; number}/10^3")
annotate("(a)", xy=(0.02, 0.94), xycoords="axes fraction")
grid()

subplot(122)
plot(2*tarray, result1/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.0 ")
plot(2*tarray, result2/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.1")
plot(2*tarray, result3/scale, label=L"\sigma_\mathrm{v}/ (\lambda \kappa)=0.2")
vlines(2*100, 0, 280000/scale, color="r", linestyle="dotted")
arrow(200, 210,100,0; length_includes_head=true, color="red", width=1, head_width=8, head_length=10)
text(210, 220, "quasi-steady-state", color="r")
#title("Time evolution for 1000 Atom clusters")
xlabel(L"\kappa t")
ylabel(L"\mathrm{photon number}/10^3")

annotate("(b)", xy=(0.02, 0.94), xycoords="axes fraction")
grid()
legend(loc="lower right")

tight_layout()